In [1]:
from Data_Tools import *
from utils import *
import logging
import time

my_seed=1
set_libraries_seeds(my_seed)# Important. Apporach reproductible results
logging.getLogger("tensorflow").setLevel(logging.ERROR)
path='../MQ2007/Fold5/'
data_vali=load_data(path+'vali.txt')
data_train=load_data(path+'train.txt')
data_test=load_data(path+'test.txt')
features_removed=['f46','f43','f42','f41',#pagerank
                  'f37','f32','f27','f22',#anchor
                  'f17','f12','f7','f2']  #anchor

Using TensorFlow backend.


Cheking the data dimensions. 

In [2]:
data=data_train.append(data_vali).append(data_test)
data.shape

(69623, 48)

Unique queries across all the folds.

In [3]:
len(data.qid.unique())

1692

# T-Student Analysis of the data

Understanding the labels 1 and 2 as the same one. We can strapolate this to a T-Test feature Selection setting

$$T(X_{i})=\frac{\left | \mu _{i1}-\mu _{i2} \right |}{\sqrt[2]{\frac{\sigma_{i1}^{2}}{N_{1}}+\frac{\sigma_{i2}^{2}}{N_{2}}}}%0$$

Then we split the data accordingly to their "relevance class"

In [4]:
data1=data[(data['relevance degree']==1)|(data['relevance degree']==2)].drop(['relevance degree', 'qid'],axis=1)
data0=data[(data['relevance degree']==0)].drop(['relevance degree', 'qid'],axis=1)

In [5]:
print(data0.shape[0]/data.shape[0],data1.shape[0]/data.shape[0])

0.7415940134725594 0.2584059865274406


There is some class, invalance. Expected for a relevance search case scenario. Most documents out there are not necessarily relevant.

In [6]:
feature_importance=pd.DataFrame()
n0=data0.shape[0] #Number of samples in the class 0
n1=data1.shape[0] #Number of samples in the class 1
lista=[]
i=1
for column in data0.columns:
    col="f"+str(i)
    u0=data0[col].mean() #mean of samples in the class 0
    var0=data0[col].var()#variance of samples in the class 0
    
    u1=data1[col].mean() #mean of samples in the class 1
    var1=data1[col].var() #variance of samples in the class 1
    
    sig=np.absolute(u1-u0)/(np.sqrt(1E-10+(var1/n1)+(var0/n0))) #1E-10 added to avoid dividing by zero
    lista.append(("f"+str(i),sig))
    i+=1
    #dt=pd.DataFrame((column,sig))

In [7]:
significance=pd.DataFrame(lista,columns=["feature","significance"]).sort_values(['significance'],ascending=False)
significance=significance.reset_index(drop=True)
significance.index+=1

In [8]:
d=[['f1', 'TF(Body)'],
 ['f2', 'TF(Anchor)'],
 ['f3', 'TF(title)'],
 ['f4', 'TF(URL)'],
 ['f5', 'TF(whole document)'],
 ['f6', 'IDF(Body)'],
 ['f7', 'IDF(Anchor)'],
 ['f8', 'IDF(title)'],
 ['f9', 'IDF(URL)'],
 ['f10', 'IDF(whole document)'],
 ['f11', 'TF*IDF(Body)'],
 ['f12', 'TF*IDF(Anchor)'],
 ['f13', 'TF*IDF(title)'],
 ['f14', 'TF*IDF(URL)'],
 ['f15', 'TF*IDF(whole document)'],
 ['f16', 'DL(Body)'],
 ['f17', 'DL(Anchor)'],
 ['f18', 'DL(title)'],
 ['f19', 'DL(URL)'],
 ['f20', 'DL(whole document)'],
 ['f21', 'BM25(Body)'],
 ['f22', 'BM25(Anchor)'],
 ['f23', 'BM25(title)'],
 ['f24', 'BM25(URL)'],
 ['f25', 'BM25(whole document)'],
 ['f26', 'LMIR.ABS(Body)'],
 ['f27', 'LMIR.ABS(Anchor)'],
 ['f28', 'LMIR.ABS(title)'],
 ['f29', 'LMIR.ABS(URL)'],
 ['f30', 'LMIR.ABS(whole document)'],
 ['f31', 'LMIR.DIR(Body)'],
 ['f32', 'LMIR.DIR(Anchor)'],
 ['f33', 'LMIR.DIR(title)'],
 ['f34', 'LMIR.DIR(URL)'],
 ['f35', 'LMIR.DIR(whole document)'],
 ['f36', 'LMIR.JM(Body)'],
 ['f37', 'LMIR.JM(Anchor)'],
 ['f38', 'LMIR.JM(title)'],
 ['f39', 'LMIR.JM(URL)'],
 ['f40', 'LMIR.JM(whole document)'],
 ['f41', 'PageRank'],
 ['f42', 'Inlink Number'],
 ['f43', 'Outlink Number'],
 ['f44', 'Number of slash URL'],
 ['f45', 'Length(URL)'],
 ['f46', 'Number of child page']]
dt_name=pd.DataFrame(d,columns=['feature','feature name'])
significance_with_names=pd.merge(significance,dt_name,on=['feature'])

In [9]:
significance_with_names

,feature,significance,feature name
0,f23,68.224340,BM25(title)
1,f39,66.209098,LMIR.JM(URL)
2,f21,56.735614,BM25(Body)
3,f37,53.467367,LMIR.JM(Anchor)
4,f40,51.988466,LMIR.JM(whole document)
5,f24,49.820794,BM25(URL)
6,f22,46.318038,BM25(Anchor)
7,f38,45.765128,LMIR.JM(title)
8,f1,19.810086,TF(Body)
9,f5,19.249713,TF(whole document)


In [10]:
significance_with_names.to_csv('../MQ2007/significance.csv',sep=' ',index=False)

> #### This is the ranking used for the report. Nonetheless, after some analysis we found that it might not be the best, given that it understands the feature space as a whole, and does not take into account the relative signifficance of a document with respect to another. In this sense, we consider it would be best to perform the same experiements again, but using the cumulative T-Score over N documents. 

# Statistical Analysis Using query-based cumulative T-Statistic

As we mentioned before, we propose a cumulative T-Statistic, taken on query-based sections of the data. This way we can have an analysis that considers the relative importance of documents given a query

In [11]:
path='../MQ2007/Fold5/'
data_vali=load_data(path+'vali.txt')
data_train=load_data(path+'train.txt')
data_test=load_data(path+'test.txt')
features_removed=['f46','f43','f42','f41',#pagerank
                  'f37','f32','f27','f22',#anchor
                  'f17','f12','f7','f2']  #anchor
data=data_train.append(data_vali).append(data_test)
queries=data['qid'].unique()
data_orig=data.copy()
Total_significance=None
total_good=0
total_zero=0
problem=0
sig_list=[]
from tqdm import tqdm_notebook as tqdm 
for query in tqdm(queries):
    data=data_orig[data_orig['qid']==query].drop(['qid'],axis=1)


    data1=data[(data['relevance degree']==1)|(data['relevance degree']==2)].drop(['relevance degree'],axis=1)
    data0=data[(data['relevance degree']==0)].drop(['relevance degree'],axis=1)
    feature_importance=pd.DataFrame()
    
    n0=data0.shape[0]
    n1=data1.shape[0]
    lista=[]
    
    i=1
    if(data0.shape[0]==0 or data1.shape[0]==0):
        #print("zero samples of one of the classes")
        total_zero+=1

    for column in data0.columns:
        
        u0=data0[column].mean()
        var0=data0[column].var()



        u1=data1[column].mean()
        var1=data1[column].var()
        
        '''
        It can be proven mathematically that the variance of a constant is zero. 
        var=E[(X-E[X])**2]... Knowing that the expectation of a constant is itself, 
        for a constant C, var=E[(c-c)**2]=E[0]=0
        
        Pandas assumes this to be a NaN, but we explicetly set it to 0 for the reasons above
        
        This happens when there is a single entry for a class in a certain query
        '''
        if data1.shape[0]==1:
            var1=0
        if data0.shape[0]==1:
            var0=0
        

        if(np.isnan(var1) or np.isnan(var0)): 
            '''
            This happens when the one of the classes is empty for a given query, in this case the
            sample should be ignored, given that there is not useful T-Statistic that helps us
            differentiate between classses
            '''
            continue
        
        sig=np.absolute(u1-u0)/(np.sqrt(((var1+0.0001)/(n1))+((var0+0.0001)/(n0))))
        lista.append((column,sig))
        i+=1
   
    significance=pd.DataFrame(lista,columns=["feature","significance"])#.sort_values(['significance'],ascending=False)
    significance=significance.reset_index(drop=True)
    significance.index+=1
    #display(significance)
    if significance.shape[0]!=0:

        total_good+=1

    if (Total_significance is None )& (significance.shape[0]!=0):
        Total_significance=significance
    else:
        if significance.shape[0]!=0:
            Total_significance['significance']+=significance.significance
            
Total_significance=Total_significance.sort_values(['significance'],ascending=False).reset_index(drop=True)
Total_significance.index+=1


In [12]:
(total_good,total_zero)

(1454, 238)

294 queries can not be taken into account in the analysis, given that they have only samples belonging to one of the classes. Thus, are irrelvant in providing information of how the features aid in separating the document types

In [13]:
dt_name=pd.DataFrame(d,columns=['feature','feature name'])
significance_with_names=pd.merge(Total_significance,dt_name,on='feature')

In [14]:
significance_with_names.sort_values(['significance'],ascending=False).reset_index(drop=True)

,feature,significance,feature name
0,f39,4791.542089,LMIR.JM(URL)
1,f23,4784.031316,BM25(title)
2,f40,4376.611644,LMIR.JM(whole document)
3,f24,4296.577702,BM25(URL)
4,f38,4211.679160,LMIR.JM(title)
5,f22,4147.343006,BM25(Anchor)
6,f21,3806.170148,BM25(Body)
7,f37,3752.793981,LMIR.JM(Anchor)
8,f31,3676.764420,LMIR.DIR(Body)
9,f32,3631.132283,LMIR.DIR(Anchor)


In [15]:
significance_with_names.to_csv('../MQ2007/significance agg.csv',sep=' ',index=False)

### Alternative using the cumulative metric. This ranking is way different than the other one. Probably would work better, as it takes into account the relative importance of documents

It is to be noted that around 294 documents are not taken into account given that they only contain one of the classes, there might be a loss of information there. Anyways we would suggest to try other LTR specific feature relevance strategies

# Correlations in the data

In [16]:
def correlation(dataset, threshold,dt_name):
    col_corr = set() # Set of all the names of deleted columns
    list_tuples=[]
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                ci=corr_matrix.columns[i]
                cj=corr_matrix.columns[j]
                namei=dt_name[dt_name['feature']==ci]['feature name'].values[0]
                namej=dt_name[dt_name['feature']==cj]['feature name'].values[0]
                list_tuples.append((namei,namej,corr_matrix.iloc[i, j]))
                
                #if colname in dataset.columns:del dataset[colname] # deleting the column from the dataset
    return list_tuples
corr_features=correlation(data_train,0.9,dt_name)

In [17]:
len(corr_features)

22

In [18]:
corr_features

[('TF(whole document)', 'TF(Body)', 0.9963428875806907),
 ('TF*IDF(Anchor)', 'TF(Anchor)', 0.9821033494408525),
 ('TF*IDF(title)', 'TF(title)', 0.9702339177006573),
 ('TF*IDF(URL)', 'TF(URL)', 0.9746006656062431),
 ('TF*IDF(whole document)', 'TF*IDF(Body)', 0.9965367432196547),
 ('DL(whole document)', 'DL(Body)', 0.9998993709201914),
 ('BM25(URL)', 'BM25(Anchor)', 0.9489305221141374),
 ('LMIR.ABS(title)', 'LMIR.ABS(Body)', 0.9746862409231666),
 ('LMIR.DIR(Body)', 'LMIR.ABS(whole document)', 0.9044272365869945),
 ('LMIR.DIR(Anchor)', 'LMIR.ABS(whole document)', 0.986223556784886),
 ('LMIR.DIR(title)', 'TF(URL)', 0.9503670628067316),
 ('LMIR.DIR(title)', 'TF*IDF(URL)', 0.9772113222775839),
 ('LMIR.DIR(whole document)', 'LMIR.DIR(URL)', 0.948106505110749),
 ('LMIR.JM(Body)', 'LMIR.DIR(URL)', 0.9783498079198181),
 ('LMIR.JM(Body)', 'LMIR.DIR(whole document)', 0.9432314854827509),
 ('LMIR.JM(Anchor)', 'BM25(Body)', 0.9749147429429361),
 ('LMIR.JM(title)', 'BM25(Anchor)', 0.9683198331477586)

We see that the anchor features removed are highly correlated to others present in the dataset. That might explain why removing them does not hinder performance in a dramatic way. 
# Variance Analysis - PCA

In [19]:
data=data_train.append(data_vali).append(data_test)

In [20]:
from sklearn.decomposition import PCA
X=data.drop(['relevance degree','qid'],axis=1)
n=1
while True:

    pca = PCA(n_components=n)

    X_pca = pca.fit(X).transform(X)

    explained_variance=np.sum(pca.explained_variance_ratio_)

    

    if(explained_variance>0.99):
        

        print("\n\nSuccess\n")
        print("n:",n)
        print("Explained Variance",explained_variance)

        break

        
    if(n%5==0):
        print("n:",n)
        print("Explained Variance",explained_variance)

    n+=1

    

#X_test_pca=pca.transform(Xtest_scaled)

n: 5
Explained Variance 0.7282686371622136
n: 10
Explained Variance 0.8710560509015395
n: 15
Explained Variance 0.9382120839443953
n: 20
Explained Variance 0.9748335818178945


Success

n: 24
Explained Variance 0.9907011082118725


99% of the variance is explained with 24 features. This is almost half the total found in the data. This leads us to think that there are some redundancies in the data, and it could be condensed in other to reduce the dimensionality of the proble. Under this scenario, the models would likely be less prone to overfitting, and be faster to train. 

We would recommend to try PCA and other dimensionality reduction mechanisms to improve performance (like manifold learning)